# Pierwsza sieć

## Importy

In [1]:
import os
import torch
from torch.utils.data import Dataset, random_split
import torchvision
from torchvision import transforms as T
import pandas as pd
from PIL import Image

## Zbiór danych

In [2]:
class FruitsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = Image.open(img_path).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [3]:
means = [0.8991, 0.7597, 0.6830]
stds = [0.2119, 0.3765, 0.4125]

In [4]:
fruit_transforms = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize(mean=means, std=stds)
])

In [5]:
dataset = FruitsDataset(csv_file="data/fruits.csv", root_dir="data/fruits", transform=fruit_transforms)

In [6]:
train_set, test_set = random_split(dataset, [6, 2], generator=torch.Generator().manual_seed(42))

## Sieć neuronowa

In [7]:
from torch import nn
from torch import optim

In [8]:
model = nn.Sequential(
    nn.Linear(256 * 256 * 3, 2), 
    nn.Softmax(dim=1)
)

In [9]:
sum([param.numel() for param in model.parameters() if param.requires_grad])

393218

In [10]:
for name, param in model.named_parameters():
    print(name, param.shape)

0.weight torch.Size([2, 196608])
0.bias torch.Size([2])


In [11]:
print(model)

Sequential(
  (0): Linear(in_features=196608, out_features=2, bias=True)
  (1): Softmax(dim=1)
)


In [12]:
softmax = nn.Softmax(dim=1)

In [13]:
x = torch.tensor([[1.0, 2.0], 
                  [4.0, 1.0]])

In [14]:
softmax(x)

tensor([[0.2689, 0.7311],
        [0.9526, 0.0474]])

In [15]:
softmax(x).sum(dim=1)

tensor([1., 1.])